**CLASIFICACIÓN MULTICLASE**

La clasificación multiclase es un tipo de problema en el aprendizaje automático donde tenemos que asignar una instancia como una imagen o texto a una de varias clases diferentes. En lugar de tener solo dos opciones como en la clasificación binaria, en la clasificación multiclase tenemos más opciones entre las cuales elegir.ff


El dataset cuenta con 17 columnas y 20000 filas, se cumple el requisito para el laboratorio n>10 y m>=20000

Valores de entrada:
xbox

ybox

width

height

onpix

xbar

ybar

x2bar

xybar

x2ybar

xy2bar

xedge

xedgey

yedgex

Valor de salida:
letter


**Importamos el drive**

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


**Importamos todas las librerias que usaremos en nuestro cuadernillo**

In [48]:
import os
import numpy as np
import pandas as pd
import sklearn as skl
from sklearn.svm import SVC
from matplotlib import pyplot
from scipy import optimize
from scipy.io import loadmat
from sklearn.model_selection import train_test_split
%matplotlib inline

In [49]:
url = "/content/gdrive/MyDrive/diamonds.csv"
dataframe = pd.read_csv(url)
print(dataframe)

       carat        cut color clarity  depth  table  price     x     y     z
0       0.23      Ideal     E     SI2   61.5   55.0    326  3.95  3.98  2.43
1       0.21    Premium     E     SI1   59.8   61.0    326  3.89  3.84  2.31
2       0.23       Good     E     VS1   56.9   65.0    327  4.05  4.07  2.31
3       0.29    Premium     I     VS2   62.4   58.0    334  4.20  4.23  2.63
4       0.31       Good     J     SI2   63.3   58.0    335  4.34  4.35  2.75
...      ...        ...   ...     ...    ...    ...    ...   ...   ...   ...
49995   0.72      Ideal     D     SI1   60.8   57.0   2757  5.75  5.76  3.50
49996   0.72       Good     D     SI1   63.1   55.0   2757  5.69  5.75  3.61
49997   0.70  Very Good     D     SI1   62.8   60.0   2757  5.66  5.68  3.56
49998   0.86    Premium     H     SI2   61.0   58.0   2757  6.15  6.12  3.74
49999   0.75      Ideal     D     SI2   62.2   55.0   2757  5.83  5.87  3.64

[50000 rows x 10 columns]


In [50]:
print(dataframe.describe())

              carat         depth         table         price             x  \
count  50000.000000  50000.000000  50000.000000  50000.000000  50000.000000   
mean       0.799444     61.753006     57.457830   3944.805440      5.734403   
std        0.475173      1.431088      2.232092   3997.938105      1.123077   
min        0.200000     43.000000     43.000000    326.000000      0.000000   
25%        0.400000     61.000000     56.000000    951.000000      4.710000   
50%        0.700000     61.800000     57.000000   2410.000000      5.700000   
75%        1.040000     62.500000     59.000000   5351.000000      6.540000   
max        5.010000     79.000000     95.000000  18823.000000     10.740000   

                  y             z  
count  50000.000000  50000.000000  
mean       5.737956      3.541056  
std        1.145579      0.707065  
min        0.000000      0.000000  
25%        4.720000      2.910000  
50%        5.710000      3.530000  
75%        6.540000      4.040000  


**Modificación de valores**



In [51]:
mapeo_Color = {'D': 0, 'E': 1, 'F': 2, 'G': 3, 'H': 4, 'I': 5, 'J': 6}
dataframe['color'] = dataframe['color'].map(mapeo_Color)

mapeo_cut = {'Ideal': 0, 'Premium': 1, 'Very Good': 2, 'Good': 3, 'Fair': 4}
dataframe['cut'] = dataframe['cut'].map(mapeo_cut)

mapeo_clarity = {'I1': 1, 'SI2': 2, 'SI1': 3, 'VS2': 4, 'VS1': 5, 'VVS2': 6, 'VVS1': 7, 'IF': 8,}
dataframe['clarity'] = dataframe['clarity'].map(mapeo_clarity)

# Imprimo el DataFrame resultante con la columna 'letter' modificada
print(dataframe)

       carat  cut  color  clarity  depth  table  price     x     y     z
0       0.23    0      1        2   61.5   55.0    326  3.95  3.98  2.43
1       0.21    1      1        3   59.8   61.0    326  3.89  3.84  2.31
2       0.23    3      1        5   56.9   65.0    327  4.05  4.07  2.31
3       0.29    1      5        4   62.4   58.0    334  4.20  4.23  2.63
4       0.31    3      6        2   63.3   58.0    335  4.34  4.35  2.75
...      ...  ...    ...      ...    ...    ...    ...   ...   ...   ...
49995   0.72    0      0        3   60.8   57.0   2757  5.75  5.76  3.50
49996   0.72    3      0        3   63.1   55.0   2757  5.69  5.75  3.61
49997   0.70    2      0        3   62.8   60.0   2757  5.66  5.68  3.56
49998   0.86    1      4        2   61.0   58.0   2757  6.15  6.12  3.74
49999   0.75    0      0        2   62.2   55.0   2757  5.83  5.87  3.64

[50000 rows x 10 columns]


**Convertir el dataset modificado en PANDAS**



In [52]:
data = dataframe.values
X, y = data[:, 2:], data[:, 1]
#Imprimo el tamaño de las caracteristicas y del label que usare
print(X.shape)
print(y.shape)

(50000, 8)
(50000,)


In [53]:
#Imprimo y para ver el array
y

array([0., 1., 3., ..., 2., 1., 0.])

**Definir variables que coincidan con el dataset**

In [54]:
input_layer_size  = 9
num_labels = 5
m = y.size

In [55]:
def calcularSigmoide(z):
    return 1.0 / (1.0 + np.exp(-z))

** calcularCosto**


In [56]:
def calcularCosto(theta, X, y, lambda_):
    # Inicializa algunos valores utiles
    m = y.size

    # convierte las etiquetas a valores enteros si son boleanos
    if y.dtype == bool:
        y = y.astype(int)

    J = 0
    grad = np.zeros(theta.shape)

    h = calcularSigmoide(X.dot(theta.T))

    temp = theta
    temp[0] = 0

    J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))

    grad = (1 / m) * (h - y).dot(X)
    # Se aplica regularizacion en la siguiente linea
    grad = grad + (lambda_ / m) * temp

    return J, grad

In [74]:
#Valor de prueba para theta
theta_t = np.array([0.5, 0.2, 0.8, 0.3, 0.1,0.4, 0.7, 0.1, 0.2], dtype=float)

# Valores de prueba para las características
# Concateno una columna de unos
#X_t = np.concatenate((np.ones((15, 17)), np.arange(1, 16).reshape(15, 1, order='F')/10.0), axis=1)

# Valores de prueba para las entradas X
X_t = np.concatenate((np.ones((50000, 1)), np.random.rand(50000, 8)), axis=1)
print(X_t)

# Valores de prueba para las clases
y_t = np.random.randint(0, 5, 50000)

# Valor de prueba para la regularización
lambda_t = 0.2

[[1.         0.4816477  0.42782895 ... 0.34892591 0.88593019 0.70271884]
 [1.         0.2962589  0.55910821 ... 0.7270817  0.95915858 0.99917336]
 [1.         0.94184094 0.39546957 ... 0.46087042 0.86406026 0.46397411]
 ...
 [1.         0.09252457 0.64056007 ... 0.17760419 0.45874657 0.43452804]
 [1.         0.09894293 0.33643631 ... 0.57919603 0.15982253 0.58771253]
 [1.         0.77402629 0.16509152 ... 0.12519397 0.87960025 0.06389801]]


In [75]:
J, grad = calcularCosto(theta_t, X_t, y_t, lambda_t)

print('Costo         : {:.6f}'.format(J))
print('Costo esperadot: 2.534819')
print('-----------------------')
print('Gradientes:')
print(' [{:.6f}, {:.6f}, {:.6f}, {:.6f}]'.format(*grad))
print('Gradientes esperados:')
print(' [0.146561, -0.548558, 0.724722, 1.398003]');

Costo         : -1.763857
Costo esperadot: 2.534819
-----------------------
Gradientes:
 [-1.140222, -0.572056, -0.563573, -0.566394]
Gradientes esperados:
 [0.146561, -0.548558, 0.724722, 1.398003]


In [76]:
def oneVsAll(X, y, num_labels, lambda_):

    m, n = X.shape
    all_theta = np.zeros((num_labels, n + 1))
    X = np.concatenate([np.ones((m, 1)), X], axis=1)

    for c in np.arange(num_labels):
        initial_theta = np.zeros(n + 1)
        options = {'maxiter': 5000}
        res = optimize.minimize(calcularCosto,
                                initial_theta,
                                (X, (y == (c + 1)), lambda_),
                                jac=True,
                                method='BFGS',
                                options=options)
        all_theta[c] = res.x
    return all_theta


In [77]:
lambda_ = 0.2
all_theta = oneVsAll(X, y, num_labels, lambda_)

<ipython-input-55-243110c07c87>:2: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1.0 + np.exp(-z))
<ipython-input-56-d22e4f31dd79>:17: RuntimeWarning: divide by zero encountered in log
  J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))
<ipython-input-56-d22e4f31dd79>:17: RuntimeWarning: divide by zero encountered in log
  J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))
<ipython-input-56-d22e4f31dd79>:17: RuntimeWarning: divide by zero encountered in log
  J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))
<ipython-input-56-d22e4f31dd79>:17: RuntimeWarning: divide by zero encountered in log
  J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))
<ipython-input-56-d22e4f31dd79>:17: RuntimeWarning: divide by zero 

In [78]:
#Imprimo las thetas
print(all_theta)

[[-6.59395020e-01  1.14004577e-02  7.21029126e-02 -3.06252385e-01
   3.35204471e-01  8.12342060e-05  7.66379306e+00 -8.77432495e+00
   1.36014103e+00]
 [-5.05689033e-01  3.47808226e-02 -5.64092623e-02 -1.39813230e-01
   1.54780275e-01  4.76606397e-05 -2.60957832e+00  1.27949117e+00
   1.83204956e+00]
 [-5.61807970e-01  3.29433225e-02 -2.46191119e-01 -4.10218375e-02
   6.46318264e-02  6.38741044e-05 -8.77674267e-01 -3.44329817e-01
   1.34044744e+00]
 [-3.99309239e-01  3.04437976e-02 -7.57899688e-01  1.03049484e-01
  -1.36714700e-01 -9.79792999e-05  1.33594397e-01 -7.33480131e-01
   1.25546061e+00]
 [-2.55973553e-04 -6.65218951e-04 -1.03610415e-03 -1.58071364e-02
  -1.47076849e-02 -1.00976587e+00 -1.46785561e-03 -1.46876509e-03
  -9.06416687e-04]]


In [79]:
def predictOneVsAll(all_theta, X):
    m = X.shape[0];
    num_labels = all_theta.shape[0]

    p = np.zeros(m)

    # Add ones to the X data matrix
    X = np.concatenate([np.ones((m, 1)), X], axis=1)
    p = np.argmax(calcularSigmoide(X.dot(all_theta.T)), axis = 1)

    return p + 1

** características del dataset para el entrenamiento (80%) y el test(20%)**



In [80]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=50)
#Imprimo el tamaño de los datos de entrenamiento y test
print(X_train.shape)
print(X_test.shape)

(40000, 8)
(10000, 8)


In [81]:

classifier =  SVC()

classifier.fit(X_train, y_train)

score = classifier.score(X_test, y_test)*100
print('Precisión del conjunto: {:.2f}%'.format(score))

Precisión del conjunto: 40.51%


In [69]:
pred_train = predictOneVsAll(all_theta, X_train)
#calculamos la precisión del entrenamiento (80%)
precision_entrenamiento = np.mean(pred_train == y_train) * 100
print('Precisión del conjunto de entrenamiento: {:.2f}%'.format(precision_entrenamiento))

XPrueba = X_test[:100].copy()
yPrueba = y_test.copy()
#concatenamos la columna de unos a XPrueba
XPrueba = np.concatenate((np.ones((100, 1)), XPrueba), axis=1)
print(XPrueba)
#Hacemos las predicciones del test (20%)
pred_prueba = np.argmax(calcularSigmoide(XPrueba.dot(all_theta.T)), axis=1)

print('Predicciones en el conjunto de prueba:')
print(pred_prueba + 1)


Precisión del conjunto de entrenamiento: 36.45%
[[1.0000e+00 4.0000e+00 2.0000e+00 6.3100e+01 5.9000e+01 1.0184e+04
  7.9600e+00 7.8800e+00 5.0000e+00]
 [1.0000e+00 3.0000e+00 2.0000e+00 6.3400e+01 5.7000e+01 1.7150e+03
  5.6200e+00 5.5800e+00 3.5500e+00]
 [1.0000e+00 1.0000e+00 4.0000e+00 6.1900e+01 6.1000e+01 8.4400e+02
  4.3200e+00 4.2800e+00 2.6600e+00]
 [1.0000e+00 3.0000e+00 4.0000e+00 6.2000e+01 5.6000e+01 1.5800e+03
  5.2800e+00 5.3000e+00 3.2800e+00]
 [1.0000e+00 4.0000e+00 7.0000e+00 6.2000e+01 5.7000e+01 2.5410e+03
  5.6600e+00 5.7000e+00 3.5200e+00]
 [1.0000e+00 5.0000e+00 5.0000e+00 6.3100e+01 5.3000e+01 9.5130e+03
  7.3400e+00 7.2500e+00 4.6000e+00]
 [1.0000e+00 1.0000e+00 2.0000e+00 6.2200e+01 5.6000e+01 4.0110e+03
  6.2000e+00 6.2400e+00 3.8700e+00]
 [1.0000e+00 0.0000e+00 3.0000e+00 5.6800e+01 6.2000e+01 2.4500e+03
  5.9300e+00 5.8900e+00 3.3700e+00]
 [1.0000e+00 2.0000e+00 4.0000e+00 6.1100e+01 5.8000e+01 8.2320e+03
  6.8200e+00 6.8600e+00 4.1800e+00]
 [1.0000e+00 3.0

<ipython-input-55-243110c07c87>:2: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1.0 + np.exp(-z))


In [72]:
# Datos para prueba (fila 28)
datos = [ [1,2,61.5,55.0,326,3.95,3.98,2.43]]
XX = np.array(datos)

XPrueba = XX.copy()
yPrueba = y_test.copy()
#concatenamos la columna de unos a XPrueba
XPrueba = np.concatenate((np.ones((1, 1)), XPrueba), axis=1)
print(XPrueba)
#Hacemos las predicciones del test (20%)
pred_prueba = np.argmax(calcularSigmoide(XPrueba.dot(all_theta.T)), axis=1)

print('Prediccion en la prueba:', pred_prueba + 1)
print('Prediccion esperada: 23')

[[  1.     1.     2.    61.5   55.   326.     3.95   3.98   2.43]]
Prediccion en la prueba: [2]
Prediccion esperada: 23
